In [6]:
import math
import numpy as np
import scipy.signal
import pandas as pd
import csv


In [7]:
def compute_air_absorption_coefficients(T, p, rel_hum, freqs):
        """
        Computes air absorption coefficients at a set of nbands equispaced frequencies in the range `[0, fs]`, based
        on the ISO 9613-1 standard. 

        A different formulation can be found in 
        `Keith Attenborough, "Sound Propagation in the Atmosphere", Springer Handbook of Acoustics`, and can be set
        by using T01 = 293.15
        The coefficients depend on the atmsopsheric temperature, pressure and relative humidity.

        Parameters
        ----------
        nbands: int
            Number of frequency bands in which to compute air absorption coefficients, by default 50

        Returns
        -------
        np.ndarray
            1D Array containing nbands air absorption coefficients, computed in dB scale, at the defined 
            set of frequencies

        """

        T0 = 293.15     # Standard room temperature T = 20deg Celsius
        T01 = 273.16    # Triple point isotherm temperature, ISO 9613-1
        T = T + 273.15
        ps0 = 1         # Standard atmospheric pressure in atm

        f = freqs # np.linspace(0, fs, num=nbands)     # Frequencies in which coeffs will be computed
        
        Csat = -6.8346 * math.pow(T01 / T, 1.261) + 4.6151
        rhosat = math.pow(10, Csat)
        H = rhosat * rel_hum * ps0 / p

        frn = (p / ps0) * math.pow(T0 / T, 0.5) * (
                9 + 280 * H * math.exp(-4.17 * (math.pow(T0 / T, 1/3.) - 1)))

        fro = (p / ps0) * (24.0 + 4.04e4 * H * (0.02 + H) / (0.391 + H))

        alpha = f * f * (
            1.84e-11 / ( math.pow(T0 / T, 0.5) * p / ps0 )
            + math.pow(T / T0, -2.5)
            * (
                0.10680 * math.exp(-3352 / T) * frn / (f * f + frn * frn)
                + 0.01278 * math.exp(-2239.1 / T) * fro / (f * f + fro * fro)
                )
            )
        
        return alpha * 20 / np.log(10)

def compute_air_absorption_filter(alpha, freqs, distance, numtaps):
        """
        Computes air absorption filter as a FIR filter with `numtaps` coefficients. The filter depends
        on the distance travelled by the sound wave between the source and the receiver.

        Parameters
        ----------
        distance : float
            Distance travelled by the sound wave, in meters
        numtaps : int
            Number of coefficients of the FIR filter

        Returns
        -------
        np.ndarray
            1D array containing `numtaps` FIR filter coefficients modelling air absorption
        """

        alpha_lin = 10 ** (-alpha * distance / 20)     # Convert coeffs in dB to linear scale
        freqs_norm = freqs / max(freqs)
        filt_coeffs = scipy.signal.firwin2(numtaps, freqs_norm, alpha_lin)
        
        return filt_coeffs

In [17]:
# Parameters
T = 20
p = 1
rel_hum = 50
fs = 8000
numbands = 50
freqs = np.linspace(0, fs / 2, numbands)
alpha = compute_air_absorption_coefficients(T, p, rel_hum, freqs)

# Distance
d = 2

# Filter
coeffs = compute_air_absorption_filter(alpha, freqs, d, 11)
np.reshape(coeffs, (1,11))

array([[ 6.38159572e-06, -7.46507571e-06,  7.34304137e-05,
        -1.69455098e-04,  1.19788090e-03,  9.97292653e-01,
         1.19788090e-03, -1.69455098e-04,  7.34304137e-05,
        -7.46507571e-06,  6.38159572e-06]])

In [19]:
coeffs

array([ 6.38159572e-06, -7.46507571e-06,  7.34304137e-05, -1.69455098e-04,
        1.19788090e-03,  9.97292653e-01,  1.19788090e-03, -1.69455098e-04,
        7.34304137e-05, -7.46507571e-06,  6.38159572e-06])

In [10]:
# Create dataset
T = np.arange(-1,1)       # Range of temperatures
p = 1
rel_hum = np.arange(0,10,5)
d = np.arange(0,2,0.5)

# alpha = np.zeros((len(T), len(rel_hum), numbands))
coeffs = np.empty((0, 11))
abs_coeffs = np.empty((0, numbands))
for i in range(len(T)):
    for j in range(len(rel_hum)):
        alpha = compute_air_absorption_coefficients(T[i],p, rel_hum[j], freqs)
        abs_coeffs = np.append(abs_coeffs, np.tile(np.reshape(alpha, (1,numbands)), (len(d),1)), axis = 0)
        for k in range(len(d)):
            coeffs = np.append(coeffs, np.reshape(compute_air_absorption_filter(alpha, freqs, d[k], numtaps=11), (1,11)), axis = 0)

print(np.shape(abs_coeffs))
# print(abs_coeffs)


(16, 8)


In [11]:
## Define a CSV File
n_abs_coeffs = 8
filt_taps = 11
header = 'id'
for i in range(0,n_abs_coeffs):
    header += f' alpha{i}'
header += ' distance'
for i in range(0,filt_taps):
    header += f' h{i}'
header = header.split()

In [12]:
file = open('data.csv', 'w', newline = '')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

In [13]:
# Create dataset
fs = 8000
freqs = np.linspace(0, fs, n_abs_coeffs)

T = np.arange(-20,40)         # Range of temperatures
p = 1
rel_hum = np.arange(0,100) # Range of relative humidities
d = np.arange(0,100,0.5)      # Range of distances

coeffs = np.empty((0, filt_taps))
abs_coeffs = np.empty((0, n_abs_coeffs))
for i in range(len(T)):
    for j in range(len(rel_hum)):
        alpha = compute_air_absorption_coefficients(T[i],p, rel_hum[j], freqs)
        id = f'T_{T[i]}p_{p}h_{rel_hum[j]}'
        
        abs_coeffs = np.append(abs_coeffs, np.tile(np.reshape(alpha, (1,n_abs_coeffs)), (len(d),1)), axis = 0)
        for k in range(len(d)):
            to_append = id + f'd_{d[k]}'
            _coeffs = compute_air_absorption_filter(alpha, freqs, d[k], numtaps=filt_taps)
            coeffs = np.append(coeffs, np.reshape(_coeffs, (1,filt_taps)), axis = 0)
            
            for e in alpha:
                to_append += f' {e}'
            to_append += f' {d[k]}'
            for e in _coeffs:
                to_append += f' {e}'
            
            file = open('data.csv', 'a', newline = '')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())

print(np.shape(coeffs))

KeyboardInterrupt: 